# Use CircleCI for CI Testing 

In [ ]:
import os
import pprint

from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain_core.prompts import PromptTemplate, load_prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import SimpleJsonOutputParser 

c:\ProgramData\miniconda3\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

True

In [ ]:
CALL_PROMPT_FILE_NAME: str = "CALL_PROMPT.yaml"
CALL_SUMMARIZATION_PROMPT_FILENAME: str = "SUMMARIZATION_PROMPT.yaml"
DEFAULT_OPENAI_MODEL_NAME: str =   "gpt-3.5-turbo"
#"gpt-4-turbo"

In [ ]:
CALL_PROMPT = """ 

Role: As a Language Model Assistant in an ambulance healthcare setting, your role is to assist with predicting call categories by accurately analyzing patient call transcripts to expedite the triage process.
    Vision:  You are extracting key information based on the Patient transcripts.
    Mission: Provide concise, relevant, and clear summaries of patient issues based on the patient transcripts, ensuring that they are classified appropriately and directed to the correct team for immediate attention. 

    While answering, for Severity Levels, you can choose a severity based on the context below:

    Low Severity:
        - Genito-Urinary conditions
        - Dental problem
        - Failed contraception
        - Palliative Care
        - Suicidal
    Medium Severity:
        - Level 2 interfacility transfer
        - Acute coronary syndrome
        - Refused Ambulance disposition
        - Potential Broken Arm/Leg
        - Covid symptoms with respiratory distress
    High Severity:
        - Major Blood Loss
        - Fitting now
        - Unconsciousness
        - Anaphylaxis
        - Cardiac arrest
        
    Provide the output in JSON format with the following keys:
        - Issue: (string) - Brief description of the patient's concern.
        - Issue Summary: (string, less than 20 words) - Concise overview of the problem.
        - Severity: ( Low , Medium , High ) - Urgency of the patient's situation.
        - Clinician to Contact: (general, support, technical) - If the transcript contains 3 or more “Not Sure” Keywords or “I don’t know” In it then output “Clinician to Contact Patient”
    
    Note: Before answering a question, ensure that the provided transcript is related to a reporting an issue or a similar scenario and not anything else apart from this. If not, then output 'Invalid' for all keys.

    Transcript:
    {Transcript}

"""

SUMMARIZATION_PROMPT = \
"""Please summarize the following text, capturing the key points and main concerns expressed in less than 10 words:

{Transcript}

Make sure to include relevant details, such as symptoms, background information, current situations, and any specific requests or questions posed.
"""

In [ ]:
# os.environ['GOOGLE_API_KEY'] = ""
# os.environ['OPENAI_API_KEY'] = ""

In [ ]:
def save_prompt(template: str, input_variables: list, file_name: str):
    """
    Save a prompt template to a file and then load it back.

    Args:
        template (str): The template string for the prompt.
        input_variables (list): List of input variables used in the template.
        file_name (str): The name of the file to save the prompt template to.

    """
    prompt = PromptTemplate(
        template=template,
        input_variables=input_variables,
    )

    prompt.save(file_name)
    print(f"Prompt template saved to {file_name}")
    
    return 0

In [ ]:
save_prompt(CALL_PROMPT, [], CALL_PROMPT_FILE_NAME)
save_prompt(SUMMARIZATION_PROMPT, [], CALL_SUMMARIZATION_PROMPT_FILENAME)

Prompt template saved to CALL_PROMPT.yaml
Prompt template saved to SUMMARIZATION_PROMPT.yaml


0

In [ ]:
class CallDataProcessor:

    def __init__(self, model_name, summarization_prompt_path, classifier_prompt_path):
        self.summarization_prompt = load_prompt(summarization_prompt_path)
        self.classifier_prompt = load_prompt(classifier_prompt_path)
        self.model_name = model_name 

    def process_input(self, input_data):
        try:


            llm = ChatOpenAI(model=self.model_name, temperature=0)
            llm_json = ChatOpenAI(model=self.model_name, temperature=0, model_kwargs={'response_format': {"type": "json_object"}})
            
            # Create the classifier and summary chains
            classifier_chain = self.classifier_prompt | llm_json | SimpleJsonOutputParser()
            classifier_result = classifier_chain.invoke(input_data)
            
            summary_chain = self.summarization_prompt | llm
            summary_result = summary_chain.invoke(input_data).content
            
            # Structure the results
            result = {
                "Transcript": input_data,
                "Summary": summary_result,
                "Issue": classifier_result['Issue'],
                "Issue Summary": classifier_result['Issue Summary'],
                "Severity": classifier_result['Severity'],
                "Clinician to Contact": classifier_result['Clinician to Contact']
            }
        
            return result
    
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [ ]:
classifier_prompt_path= CALL_PROMPT_FILE_NAME
summarization_prompt_path = CALL_SUMMARIZATION_PROMPT_FILENAME

processor = CallDataProcessor(DEFAULT_OPENAI_MODEL_NAME, summarization_prompt_path, classifier_prompt_path)

In [ ]:
input = "Please hurry, she's unconscious on the floor! I tried checking her pulse, but there's nothing. We've already called 911, but she needs help now! I think she might have hit her head when she fell. Is there anything we can do while we wait for the ambulance?"

In [ ]:
llm_response = processor.process_input(input)

llm_response

{'Transcript': "Please hurry, she's unconscious on the floor! I tried checking her pulse, but there's nothing. We've already called 911, but she needs help now! I think she might have hit her head when she fell. Is there anything we can do while we wait for the ambulance?",
 'Summary': 'Urgent situation: unconscious woman, no pulse, possible head injury.',
 'Issue': 'Unconsciousness',
 'Issue Summary': 'Patient is unconscious and unresponsive.',
 'Severity': 'High',
 'Clinician to Contact': 'Invalid'}

# **Write Files**

In [ ]:
%%writefile requirements.txt

langchain
langchain-community
langchain-google-genai
langchain-openai
langchain-together

numpy
pandas

pillow
pydantic
pydantic-settings
python-dotenv
python-multipart
sentence-transformers
starlette
tiktoken
uvicorn

Overwriting requirements.txt


In [ ]:
%%writefile application.py

import os
import pprint

from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain_core.prompts import PromptTemplate, load_prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import SimpleJsonOutputParser 
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

# Parameters

CALL_PROMPT_FILE_NAME: str = "CALL_PROMPT.yaml"
CALL_SUMMARIZATION_PROMPT_FILENAME: str = "SUMMARIZATION_PROMPT.yaml"
DEFAULT_OPENAI_MODEL_NAME: str =   "gpt-3.5-turbo"
#"gpt-4-turbo"


# Prompts

CALL_PROMPT = """ 

    Role: As a Language Model Assistant in an ambulance healthcare setting, your role is to assist with predicting call categories by accurately analyzing patient call transcripts to expedite the triage process.
    Vision:  You are extracting key information based on the Patient transcripts.
    Mission: Provide concise, relevant, and clear summaries of patient issues based on the patient transcripts, ensuring that they are classified appropriately and directed to the correct team for immediate attention. 

    While answering, for Severity Levels, you can choose a severity based on the context below:

    Low Severity:
        - Genito-Urinary conditions
        - Dental problem
        - Failed contraception
        - Palliative Care
        - Suicidal
    Medium Severity:
        - Level 2 interfacility transfer
        - Acute coronary syndrome
        - Refused Ambulance disposition
        - Potential Broken Arm/Leg
        - Covid symptoms with respiratory distress
    High Severity:
        - Major Blood Loss
        - Fitting now
        - Unconsciousness
        - Anaphylaxis
        - Cardiac arrest
        
    Provide the output in JSON format with the following keys:
        - Issue: (string) - Brief description of the patient's concern.
        - Issue Summary: (string, less than 20 words) - Concise overview of the problem.
        - Severity: ( Low , Medium , High ) - Urgency of the patient's situation.
        - Clinician to Contact: (general, support, technical) - If the transcript contains 3 or more “Not Sure” Keywords or “I don’t know” In it then output “Clinician to Contact Patient”
    
    Note: Before answering a question, ensure that the provided transcript is related to a reporting an issue or a similar scenario and not anything else apart from this. If not, then output 'Invalid' for all keys.

    Transcript:
    {Transcript}

"""

SUMMARIZATION_PROMPT = \
"""Please summarize the following text, capturing the key points and main concerns expressed in less than 10 words:

{Transcript}

Make sure to include relevant details, such as symptoms, background information, current situations, and any specific requests or questions posed.
"""


# Helper Functions

def save_prompt(template: str, input_variables: list, file_name: str):
    """
    Save a prompt template to a file and then load it back.

    Args:
        template (str): The template string for the prompt.
        input_variables (list): List of input variables used in the template.
        file_name (str): The name of the file to save the prompt template to.

    """
    prompt = PromptTemplate(
        template=template,
        input_variables=input_variables,
    )

    prompt.save(file_name)
    print(f"Prompt template saved to {file_name}")
    
    return 0


save_prompt(CALL_PROMPT, [], CALL_PROMPT_FILE_NAME)
save_prompt(SUMMARIZATION_PROMPT, [], CALL_SUMMARIZATION_PROMPT_FILENAME)


class CallDataProcessor:

    def __init__(self, model_name, summarization_prompt_path, classifier_prompt_path):
        self.summarization_prompt = load_prompt(summarization_prompt_path)
        self.classifier_prompt = load_prompt(classifier_prompt_path)
        self.model_name = model_name 

    def process_input(self, input_data):
        try:

            llm = ChatOpenAI(model=self.model_name, temperature=0)
            llm_json = ChatOpenAI(model=self.model_name, temperature=0, model_kwargs={'response_format': {"type": "json_object"}})
            
            # Create the classifier and summary chains
            classifier_chain = self.classifier_prompt | llm_json | SimpleJsonOutputParser()
            classifier_result = classifier_chain.invoke(input_data)
            
            summary_chain = self.summarization_prompt | llm
            summary_result = summary_chain.invoke(input_data).content
            
            # Structure the results
            result = {
                "Transcript": input_data,
                "Summary": summary_result,
                "Issue": classifier_result['Issue'],
                "Issue Summary": classifier_result['Issue Summary'],
                "Severity": classifier_result['Severity'],
                "Clinician to Contact": classifier_result['Clinician to Contact']
            }
        
            return result
    
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
classifier_prompt_path= CALL_PROMPT_FILE_NAME
summarization_prompt_path = CALL_SUMMARIZATION_PROMPT_FILENAME

processor = CallDataProcessor(DEFAULT_OPENAI_MODEL_NAME, summarization_prompt_path, classifier_prompt_path)


# Use as Example

# input_text = "A woman at the library collapsed and isn't breathing. We need an ambulance right away!"
# processor.process_input(input_text)

Overwriting application.py


---
**LLM TESTING**

---

- Examples

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

def evaluation_chain(
    input_data,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    output_parser=StrOutputParser()
):
    eval_system_prompt = """You are an assistant that evaluates whether an input containing a patient transcript and related information has all required fields present and non-empty.
    The input data should include the following keys: Transcript, Summary, Issue, Issue Summary, Severity, Clinician to Contact.
    """

    eval_user_message = f"""Please evaluate the following input data to ensure all required fields are present and contain valid, non-empty data:
    Here is the data:
    [BEGIN DATA]
    ************
    Transcript: {input_data.get('Transcript', '')}
    Summary: {input_data.get('Summary', '')}
    Issue: {input_data.get('Issue', '')}
    Issue Summary: {input_data.get('Issue Summary', '')}
    Severity: {input_data.get('Severity', '')}
    Clinician to Contact: {input_data.get('Clinician to Contact', '')}
    ************
    [END DATA]

    Check if the following fields are present and contains valid, non-empty data:
    - Transcript
    - Summary
    - Issue
    - Issue Summary
    - Severity
    - Clinician to Contact


    Respond with 'Y' if all fields are present and non-empty, 'N' if any field is missing, empty, or if the field contains the keyword "Invalid".
    """

    eval_prompt = ChatPromptTemplate.from_messages([
        ("system", eval_system_prompt),
        ("human", eval_user_message),
    ])

    return eval_prompt | llm | output_parser

In [ ]:
%%writefile prompt_testing.py

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

def evaluation_chain(
    input_data,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    output_parser=StrOutputParser()
):
    eval_system_prompt = """You are an assistant that evaluates whether an input containing a patient transcript and related information has all required fields present and non-empty.
    The input data should include the following keys: Transcript, Summary, Issue, Issue Summary, Severity, Clinician to Contact.
    """

    eval_user_message = f"""Please evaluate the following input data to ensure all required fields are present and contain valid, non-empty data:
    Here is the data:
    [BEGIN DATA]
    ************
    Transcript: {input_data.get('Transcript', '')}
    Summary: {input_data.get('Summary', '')}
    Issue: {input_data.get('Issue', '')}
    Issue Summary: {input_data.get('Issue Summary', '')}
    Severity: {input_data.get('Severity', '')}
    Clinician to Contact: {input_data.get('Clinician to Contact', '')}
    ************
    [END DATA]

    Check if the following fields are present and contains valid, non-empty data:
    - Transcript
    - Summary
    - Issue
    - Issue Summary
    - Severity
    - Clinician to Contact


    Respond with 'Y' if all fields are present and non-empty, 'N' if any field is missing, empty, or if the field contains the keyword "Invalid".
    """

    eval_prompt = ChatPromptTemplate.from_messages([
        ("system", eval_system_prompt),
        ("human", eval_user_message),
    ])

    return eval_prompt | llm | output_parser

Writing prompt_testing.py


In [ ]:
bad_example = {
    'Transcript': "",
    'Summary': 'A woman at the library has collapsed and is currently not breathing, prompting an urgent call for an ambulance.',
    'Issue': 'A woman collapsed and is not breathing at the library.',
    'Issue Summary': 'Woman collapsed, not breathing, requires immediate help.',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(bad_example)
response = eval_chain.invoke({})
print(response)
assert response == "N", f"Expected response to be 'N', but it was not."


N


In [ ]:
bad_example2 = {
    'Transcript': "Invalid",
    'Summary': 'A woman at the library has collapsed and is currently not breathing, prompting an urgent call for an ambulance.',
    'Issue': 'A woman collapsed and is not breathing at the library.',
    'Issue Summary': 'Woman collapsed, not breathing, requires immediate help.',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(bad_example2)
response2 = eval_chain.invoke({})
print(response2)
assert response == "N", f"Expected response to be 'N', but it was not."


N


In [ ]:
good_example = {
    'Transcript': "A woman at the library collapsed and isn't breathing. We need an ambulance right away!",
    'Summary': 'A woman at the library has collapsed and is currently not breathing, prompting an urgent call for an ambulance.',
    'Issue': 'A woman collapsed and is not breathing at the library.',
    'Issue Summary': 'Woman collapsed, not breathing, requires immediate help.',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(good_example)
response = eval_chain.invoke({})
print(response)
assert response == "Y", f"Expected response to be 'N', but it was not."


Y


In [ ]:
good_example = \
{'Transcript': "Please hurry, she's unconscious on the floor! I tried checking her pulse, but there's nothing. We've already called 911, but she needs help now! I think she might have hit her head when she fell. Is there anything we can do while we wait for the ambulance?",
'Summary': 'Urgent situation: unconscious woman, no pulse, possible head injury, awaiting ambulance.',
'Issue': 'Unconscious patient, possible head injury',
'Issue Summary': 'Unconscious with potential head injury',
'Severity': 'High',
'Clinician to Contact': 'general'}
eval_chain = evaluation_chain(good_example)
response = eval_chain.invoke({})
print(response)
assert response == "Y", f"Expected response to be 'N', but it was not."

Y


In [ ]:
%%writefile test.py

import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

from prompt_testing import evaluation_chain

bad_example = {
    'Transcript': "",
    'Summary': 'A woman at the library has collapsed and is currently not breathing, prompting an urgent call for an ambulance.',
    'Issue': 'A woman collapsed and is not breathing at the library.',
    'Issue Summary': 'Woman collapsed, not breathing, requires immediate help.',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(bad_example)
response = eval_chain.invoke({})
print(response)
assert response == "N", f"Expected response to be 'N', but it was not."

bad_example2 = {
    'Transcript': "Invalid",
    'Summary': 'A woman at the library has collapsed and is currently not breathing, prompting an urgent call for an ambulance.',
    'Issue': 'A woman collapsed and is not breathing at the library.',
    'Issue Summary': 'Woman collapsed, not breathing, requires immediate help.',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(bad_example2)
response2 = eval_chain.invoke({})
print(response2)
assert response2 == "N", f"Expected response to be 'N', but it was not."

good_example1 = {
    'Transcript': "A woman at the library collapsed and isn't breathing. We need an ambulance right away!",
    'Summary': 'A woman at the library has collapsed and is currently not breathing, prompting an urgent call for an ambulance.',
    'Issue': 'A woman collapsed and is not breathing at the library.',
    'Issue Summary': 'Woman collapsed, not breathing, requires immediate help.',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(good_example1)
response1 = eval_chain.invoke({})
print(response1)
assert response1 == "Y", f"Expected response to be 'Y', but it was not."

good_example2 = {
    'Transcript': "Please hurry, she's unconscious on the floor! I tried checking her pulse, but there's nothing. We've already called 911, but she needs help now! I think she might have hit her head when she fell. Is there anything we can do while we wait for the ambulance?",
    'Summary': 'Urgent situation: unconscious woman, no pulse, possible head injury, awaiting ambulance.',
    'Issue': 'Unconscious patient, possible head injury',
    'Issue Summary': 'Unconscious with potential head injury',
    'Severity': 'High',
    'Clinician to Contact': 'general'
}

eval_chain = evaluation_chain(good_example2)
response2 = eval_chain.invoke({})
print(response2)
assert response2 == "Y", f"Expected response to be 'Y', but it was not."


N
N
Y
Y
